# Space

In [ ]:
import os
import sys
import logging
import pandas as pd 
from pprint import pprint 

from datasets import disable_caching, disable_progress_bars
from IPython.display import display, HTML

pd.set_option('display.max_columns', None)
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; os.chdir(WORKSPACE_PATH)

from proj_space import SPACE
sys.path.append(SPACE['CODE_FN']); SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH

disable_caching()
disable_progress_bars()
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='[%(levelname)s:%(asctime)s:(%(filename)s@%(lineno)d %(name)s)]: %(message)s')

# Inference Entry

In [ ]:
from recfldtkn.record_base.cohort import CohortFn, Cohort
from config.config_record.Cohort import CohortName_to_OneCohortArgs


CohortName = '20241013_InferencePttSampleV0'
CohortName_list = [CohortName]
CohortName_list_for_inference = CohortName_list
OneEntryArgs_items_for_inference = ['Input_Part']

Record_Proc_Config = {
    'save_data': False, 
    'load_data':False, 
    'via_method': 'df'
}

Case_Proc_Config = {
    'max_trigger_case_num': None, 
    'use_task_cache': True, 
    'caseset_chunk_size': 50000,
    'save_data': False, 
    'load_data': False, 
    'load_casecollection': False,
    'via_method': 'df',
    'n_cpus': 1, 
    'batch_size': None,  
}


In [ ]:
# get an Inference_Entry
################
# replace this part with the API input. 
OneCohort_Args = CohortName_to_OneCohortArgs[CohortName]


Source2CohortName = OneCohort_Args['Source2CohortName']
cohort_fn = CohortFn(Source2CohortName, SPACE)
cohort = Cohort(OneCohort_Args, SPACE, cohort_fn)
cohort.setup_fn(cohort_fn)
cohort.initialize_cohort()

# Get Inference_Entry
SourceFile_List = cohort.SourceFile_List
OneCohort_Args = cohort.OneCohort_Args
get_RawName_from_SourceFile = cohort.get_RawName_from_SourceFile
get_InferenceEntry = cohort.cohort_fn.get_InferenceEntry
Inference_Entry = get_InferenceEntry(OneCohort_Args, 
                                     SourceFile_List, 
                                     get_RawName_from_SourceFile)

pprint(Inference_Entry['template_form'], sort_dicts=False, compact=True)
########################

In [ ]:
print([i for i in Inference_Entry])

In [ ]:
from recfldtkn.record_base.record import Record, RecordFn

TriggerRecordName = 'CGM5Min'
Rec_Fn = RecordFn(TriggerRecordName, SPACE)

pprint(Rec_Fn.RawName_to_RawConfig)

In [ ]:
def TriggerFn_CGM5MinEntry_v1211(inference_form):
    ########### ------ build the df_case and save to the Inference_Entry ------ ###########
    TriggerName = 'CGM5MinEntry'

    DBTableName = 'ElogBGEntry'
    df_recraw = pd.DataFrame(inference_form[DBTableName])
    case = df_recraw.iloc[-1]
    PatientID = case['PatientID']
    TimezoneOffset = case['TimezoneOffset']
    ObsDT = case['ObservationDateTime'] #### this time should be adjusted with 
    ObsDTLocal = pd.to_datetime(ObsDT) + pd.Timedelta(TimezoneOffset, unit='m')
    case = {
        'PatientID': PatientID,
        'ObsDT': ObsDTLocal, # must be local, because the processed data to model is local as well.
    }

    CaseTriggerList = [case]
    TriggerName_to_CaseTriggerList = {
        TriggerName: CaseTriggerList,
    }
    return TriggerName_to_CaseTriggerList

inferece_form_name = [i for i in Inference_Entry if 'inference_form' in i][0]
inference_form = Inference_Entry[inferece_form_name]
TriggerName_to_CaseTriggerList = TriggerFn_CGM5MinEntry_v1211(inference_form)
TriggerName_to_dfCaseTrigger = {TriggerName: pd.DataFrame(CaseTriggerList) for TriggerName, CaseTriggerList in TriggerName_to_CaseTriggerList.items()}
Inference_Entry['TriggerName_to_dfCaseTrigger'] = TriggerName_to_dfCaseTrigger
TriggerName_to_dfCaseTrigger

# Config

In [ ]:
MODEL_ENDPOINT = 'vTestCGMFull'
SPACE['MODEL_ENDPOINT'] = MODEL_ENDPOINT
pprint(SPACE, sort_dicts=False, compact=True)

# Step 1: Model_Base from a Model Version

In [ ]:
from recfldtkn.model_base.model_base import Model_Base
from nn import load_model_instance_from_nn

ModelEndpoint_Path = os.path.join(SPACE['MODEL_ROOT'], SPACE['MODEL_ENDPOINT'])
model_base = Model_Base(
    ModelEndpoint_Path = ModelEndpoint_Path,
    load_model_instance_from_nn = load_model_instance_from_nn,
    SPACE = SPACE,
)

In [ ]:
pprint(model_base.ModelArtifactName_to_ModelInfo, sort_dicts=False, compact=True)

for model_artifact_name, ModelInfo in model_base.ModelArtifactName_to_ModelInfo.items():
    # model_instance 
    print(model_artifact_name)
    model_artifact = ModelInfo['model_artifact']
    print({k: len(v['input_ids']['tid2tkn']) for k, v in model_artifact.aidata.CF_to_CFvocab.items()})
    # pprint(model_instance.aidata.EntryArgs['Input_FullArgs'])
    print()

# Step 2. InfoSettings

In [ ]:
from inference.utils_inference import get_complete_InfoSettings

################### The things you need to provide in advance. 
# InputCFArgs_ForInference = None
InputCFArgs_ForInference = ['cf.TargetCGM_Bf24H'] # to remove in the future
###################

InfoSettings = get_complete_InfoSettings(model_base, CohortName_list, InputCFArgs_ForInference)

print([i for i in InfoSettings])
pprint(InfoSettings['TriggerCaseBaseName_to_TriggerCaseBaseArgs'], sort_dicts=False, compact=True)

# Step 3. AIData 

In [ ]:
from recfldtkn.aidata_base.aidata_base import AIData_Base   

OneAIDataName_to_OneAIDataArgs = InfoSettings['OneAIDataName_to_OneAIDataArgs']
pprint({k:v['OneEntryArgs'] for k, v in OneAIDataName_to_OneAIDataArgs.items()}, sort_dicts=False, compact=True)  

In [ ]:
OneAIDataName = [i for i in OneAIDataName_to_OneAIDataArgs][0]
OneAIDataArgs = OneAIDataName_to_OneAIDataArgs[OneAIDataName]
[i for i in OneAIDataArgs['OneEntryArgs']]

In [ ]:
[i for i in OneAIDataName_to_OneAIDataArgs]

In [ ]:
aidata_base = AIData_Base(
    OneAIDataName_to_OneAIDataArgs = OneAIDataName_to_OneAIDataArgs,
    OneEntryArgs_items_for_inference = OneEntryArgs_items_for_inference,
    CohortName_list_for_inference = CohortName_list_for_inference, 
    SPACE = SPACE, 
)

In [ ]:
OneAIDataName = aidata_base.get_AIDataName_list()[0]
OneAIDataArgs = aidata_base.get_OneAIDataArgs_from_OneAIDataName(OneAIDataName)

OneEntryArgs = OneAIDataArgs['OneEntryArgs']
pprint(OneEntryArgs, sort_dicts=False, compact=True)
aidata = aidata_base.get_aidata_from_OneAIDataName(OneAIDataName)
aidata

In [ ]:
# aidata_base.get_AIDataHashName_list()
Name_to_Data = aidata.Name_to_Data
[i for i in Name_to_Data]   

# Step 4: CF, HRF, TriggerCaseBaseName_to_CohortNameList

In [ ]:
OneAIDataArgs['OneEntryArgs']

In [ ]:
from recfldtkn.case_base.caseutils import get_ROCOGammePhiInfo_from_CFList

Case_Args_Settings = InfoSettings['Case_Args_Settings']
CF_to_CFArgs = Case_Args_Settings['CF_to_CFArgs']
TagCF_to_TagCFArgs = Case_Args_Settings.get('TagCF_to_TagCFArgs', {})

CF_list_ForInference = InfoSettings['CF_list_ForInference'] # (aidata_base, CF_to_CFArgs, TagCF_to_TagCFArgs)
pprint(CF_list_ForInference, sort_dicts=False)

ROCOGammaPhiInfo = get_ROCOGammePhiInfo_from_CFList(CF_list_ForInference, CF_to_CFArgs)
HumanRecordRecfeat_Args = ROCOGammaPhiInfo['HumanRecordRecfeat_Args']
pprint(HumanRecordRecfeat_Args, sort_dicts=False)


TriggerCaseBaseName_List = list(set([v['TriggerCaseBaseName'] for k, v in OneAIDataName_to_OneAIDataArgs.items()]))
TriggerCaseBaseName_to_CohortNameList = {}
for TriggerCaseBaseName in TriggerCaseBaseName_List:
    TriggerCaseBaseName_to_CohortNameList[TriggerCaseBaseName] = CohortName_list

pprint(TriggerCaseBaseName_to_CohortNameList, sort_dicts=False)

# Step 5: load AIData Model InfoSettings

In [ ]:
ModelEndpoint_Path = ModelEndpoint_Path
print(ModelEndpoint_Path)

In [ ]:
InputCFArgs_ForInference = InputCFArgs_ForInference
print(InputCFArgs_ForInference)

In [ ]:
InferenceArgs = {
    'GEN_Args': {
        'num_first_tokens_for_gen': 289,
        'max_new_tokens': 24,
        'do_sample': False,
        'items_list': ['hist', 'pred', 'logit_scores']}
    }

print(InferenceArgs)

In [ ]:
INF_CohortName = CohortName 
print(INF_CohortName)
INF_OneCohortArgs = CohortName_to_OneCohortArgs[INF_CohortName]
print(INF_OneCohortArgs)

Package_Settings = {
    'INF_CohortName': INF_CohortName,
    'INF_OneCohortArgs': INF_OneCohortArgs,
    'Record_Proc_Config': Record_Proc_Config,
    'Case_Proc_Config': Case_Proc_Config,
    'OneEntryArgs_items_for_inference': OneEntryArgs_items_for_inference,
    'get_ROCOGammePhiInfo_from_CFList': get_ROCOGammePhiInfo_from_CFList,
    'load_model_instance_from_nn': load_model_instance_from_nn,
    'Model_Base': Model_Base,
    'AIData_Base': AIData_Base,
}

In [ ]:
from inference.utils_inference import load_AIData_Model_InfoSettings

Context = load_AIData_Model_InfoSettings(
    ModelEndpoint_Path = ModelEndpoint_Path, 
    InputCFArgs_ForInference = InputCFArgs_ForInference,
    InferenceArgs = InferenceArgs,
    SPACE = SPACE, 
    **Package_Settings
)


In [ ]:
model_base = Context['model_base']
aidata_base = Context['aidata_base']
InfoSettings = Context['InfoSettings']

# Step 6: record_base

In [ ]:
print([i for i in Inference_Entry])

In [ ]:
from datetime import datetime 
from recfldtkn.record_base import Record_Base
from config.config_record.Cohort import CohortName_to_OneCohortArgs


TriggerCaseBaseName_to_TriggerCaseBaseArgs = InfoSettings['TriggerCaseBaseName_to_TriggerCaseBaseArgs']

s = datetime.now()
record_base = Record_Base(CohortName_list, 
                            HumanRecordRecfeat_Args,
                            CohortName_to_OneCohortArgs,
                            SPACE = SPACE, 
                            Inference_Entry = Inference_Entry,
                            Record_Proc_Config = Record_Proc_Config,
                            )
e = datetime.now()
du1 = e-s

In [ ]:
record_base.CohortName_to_OneCohortRecordBase

In [ ]:
one_cohort_recordbase = record_base.CohortName_to_OneCohortRecordBase[CohortName]
one_cohort_recordbase

In [ ]:
one_cohort_recordbase.TriggerName_to_dfCaseTrigger

# Step 7: case_base

In [ ]:
from recfldtkn.case_base import Case_Base

# there is no P-Demo. 
s = datetime.now()
case_base = Case_Base(
    record_base = record_base, 
    TriggerCaseBaseName_to_CohortNameList = TriggerCaseBaseName_to_CohortNameList, 
    TriggerCaseBaseName_to_TriggerCaseBaseArgs = TriggerCaseBaseName_to_TriggerCaseBaseArgs,
    Case_Proc_Config = Case_Proc_Config,
    Case_Args_Settings = Case_Args_Settings, 
)
e = datetime.now()
du2 = e-s

In [ ]:
case_base.TriggerCaseBaseName_to_CFtoCFvocab


for TriggerCaseBaseName, CF_to_CFVocab in case_base.TriggerCaseBaseName_to_CFtoCFvocab.items():
    print(TriggerCaseBaseName)
    print({k: len(v['input_ids']['tid2tkn']) for k, v in CF_to_CFVocab.items()})
    print()

In [ ]:
case_base.TriggerCaseBaseName_to_CaseSetNameToCaseset

In [ ]:
from IPython.display import display

case_base.TriggerCaseBaseName_to_CFtoCFvocab

for TriggerCaseBaseName, CaseSetName_to_CaseSet in case_base.TriggerCaseBaseName_to_CaseSetNameToCaseset.items():
    print(TriggerCaseBaseName)
    for CaseSetName, caseset in CaseSetName_to_CaseSet.items():
        print(CaseSetName)
        display(caseset.df_case)
        display(caseset.ds_case)

In [ ]:
len(caseset.ds_case.iloc[0]['cf.TargetCGM_Bf24H--input_ids'])

# Step 8: aidata_base

In [ ]:
case_base.TriggerCaseBaseName_to_CaseSetNameToCaseset

In [ ]:
s = datetime.now()
aidata_base.update_CaseBase(case_base)
e = datetime.now()
du3 = e-s

In [ ]:
AIDataName_list = aidata_base.get_AIDataName_list()
AIDataName_list

In [ ]:
OneAIDataName = AIDataName_list[0]
OneAIDataName

In [ ]:
aidata_base.case_base.TriggerCaseBaseName_to_CaseSetNameToCaseset

In [ ]:
aidata = aidata_base.get_aidata_from_OneAIDataName(OneAIDataName)

In [ ]:
aidata.INPUT_CFs

In [ ]:
aidata_base.get_AIDataName_list()

In [ ]:
for OneAIDataHash, aidata in aidata_base.OneAIDataHash_to_AIData.items():
    print(OneAIDataHash)
    for Name, Data in aidata.Name_to_Data.items():
        ds_tfm = Data['ds_tfm']
        print(Name)
        print(ds_tfm)
        print()

# Step 9: model_base

In [ ]:
##########################################
InferenceArgs = {
    'GEN_Args': {
        'num_first_tokens_for_gen': 289,
        'max_new_tokens': 24,
        'do_sample': False,
        'items_list': ['hist', 'pred', 'logit_scores']}
    }
##########################################

In [ ]:
[i for i in model_base.ModelArtifactName_to_ModelInfo]

In [ ]:
s = datetime.now()
# model_base.update_AIDataBase(aidata_base, update_df_modelinstance = False)

######## update te aidata_base to model_base ########
model_base.aidata_base = aidata_base


ModelArtifactName_to_ModelInfo = model_base.ModelArtifactName_to_ModelInfo
ModelArtifactName_to_Inference = {}
for model_artifact_name, ModelInfo in ModelArtifactName_to_ModelInfo.items():
    # OneAIDataName = modelinstanceinfo['OneAIDataName']
    # model_instance = modelinstanceinfo['model_instance']
    model_artifact = ModelInfo['model_artifact']

    OneAIDataName = model_artifact.aidata.OneAIDataName
    aidata = aidata_base.get_aidata_from_OneAIDataName(OneAIDataName)
    Name = [i for i in aidata.Name_to_Data][0]
    Data = aidata.Name_to_Data[Name]
    logger.info('Start inference process....')
    inference = model_artifact.inference(Data, InferenceArgs)
    # model_instance.model_checkpoint_name
    ModelArtifactName_to_Inference[model_artifact.model_artifact_name] = inference
e = datetime.now()
du4 = e-s

total_time = du1 + du2 + du3 + du4
print('record_base:', du1)
print('case_base:', du2)
print('aidata_base and model_base update:', du3)
print('model_infernece:', du4)
print('total_time:', total_time)

In [ ]:
pprint(ModelArtifactName_to_Inference)

In [ ]:
model_artifact_name = [i for i in ModelArtifactName_to_Inference][0]

InferenceInfo = ModelArtifactName_to_Inference[model_artifact_name]

In [ ]:
InferenceInfo['df_case_eval'].iloc[0]['hist']

In [ ]:
InferenceInfo['df_case_eval'].iloc[0]['pred']

# Step 10: pipeline_inference_for_modelbase

In [ ]:
from inference.utils_inference import pipeline_inference_for_modelbase

inference_results = pipeline_inference_for_modelbase(Inference_Entry,
                                                    Record_Base,
                                                    Case_Base,
                                                    aidata_base, 
                                                    model_base,
                                                    InfoSettings, 
                                                    SPACE)

In [ ]:
pprint(inference_results, sort_dicts=False)

In [ ]:
ModelArtifactName_to_Inference = inference_results['ModelArtifactName_to_Inference']
ModelArtifactName_to_Inference

In [ ]:
ModelArtifactName = [i for i in ModelArtifactName_to_Inference][0]
InferenceInfo = ModelArtifactName_to_Inference[ModelArtifactName]
InferenceInfo

In [ ]:
df = InferenceInfo['df_case_eval']
df

# Step 11: PostFn

In [ ]:
from datetime import timezone

def PostFn_WithCGMPred_v1210(ModelArtifactName_to_Inference, SPACE):
    # output = {"models": [], "status": {"code": 200, "message": "Success"}}
    assert len(ModelArtifactName_to_Inference) == 1, "Only one model is supported for now"

    ModelArtifactName = [i for i in ModelArtifactName_to_Inference][0]
    InferenceInfo = ModelArtifactName_to_Inference[ModelArtifactName]
    df = InferenceInfo['df_case_eval']

    hist = df.iloc[0]['hist']
    pred = df.iloc[0]['pred']
    output = {"hist": hist, "pred": pred}
    return output



PostFn = PostFn_WithCGMPred_v1210

In [ ]:
output = PostFn(ModelArtifactName_to_Inference, SPACE)

pprint(output, sort_dicts=False)